In [ ]:
from llamabot import SimpleBot
from llamabot.bot_library.diffbot import diffbot, get_github_diff

In [ ]:
url = "https://github.com/pyjanitor-devs/pyjanitor/pull/1262"


# print(get_github_diff(url))

In [ ]:
from llamabot.bot_library.diffbot import (
    summarize,
    describe_advantages,
    suggest_improvements,
    diffbot,
)


diff = get_github_diff("https://github.com/pyjanitor-devs/pyjanitor/pull/1262")

diffbot(describe_advantages(diff))

In [ ]:
diffbot(suggest_improvements(diff))

In [ ]:
asdfasdfadsf